Path to notebook: [https://www.github.com/microsoft/presidio/blob/main/docs/samples/python/example_pdf_annotation.ipynb](https://www.github.com/microsoft/presidio/blob/main/docs/samples/python/example_pdf_annotation.ipynb)

# Annotating PII in a PDF

This sample takes a PDF as an input, extracts the text, identifies PII using Presidio and annotates the PII using highlight annotations.

## Prerequisites
Before getting started, make sure the following packages are installed. For detailed documentation, see the [installation docs](https://microsoft.github.io/presidio/installation).

Install from PyPI:

In [1]:
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg
!pip install pdfminer.six
!pip install pikepdf

  Obtaining dependency information for presidio_analyzer from https://files.pythonhosted.org/packages/df/38/550844e8a797cd8101ea6b3911a0222ba9d02938ea4ea5bd7c08ea84f1b7/presidio_analyzer-2.2.353-py3-none-any.whl.metadata
  Obtaining dependency information for spacy<4.0.0,>=3.4.4 from https://files.pythonhosted.org/packages/60/37/f8b6807426300c4cb9aee6a04979df2ddaeb02f2579caf4599232fbab8bd/spacy-3.7.4-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for tldextract from https://files.pythonhosted.org/packages/fc/6d/8eaafb735b39c4ab3bb8fe4324ef8f0f0af27a7df9bb4cd503927bd5475d/tldextract-5.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for phonenumbers<9.0.0,>=8.12 from https://files.pythonhosted.org/packages/c6/17/cccaee08b0eedf016b10ef8d3322595df46365959e38c3086eb38d719080/phonenumbers-8.13.32-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8

In [2]:
# For Presidio
from presidio_analyzer import AnalyzerEngine, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig

# For console output
from pprint import pprint

# For extracting text
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLine

# For updating the PDF
from pikepdf import Pdf, AttachedFileSpec, Name, Dictionary, Array

## Analyze the text in the PDF

To extract the text from the PDF, we use the pdf miner library. We extract the text from the PDF at the text container level. This is roughly equivalent to a paragraph. 

We then use Presidio Analyzer to identify the PII and it's location in the text.

The Presidio analyzer is using pre-defined entity recognizers, and offers the option to create custom recognizers.


In [9]:
analyzer = AnalyzerEngine()

analyzed_character_sets = []

for page_layout in extract_pages("./Demodata_PIIText.pdf"):
    for text_container in page_layout:
        if isinstance(text_container, LTTextContainer):

            # The element is a LTTextContainer, containing a paragraph of text.
            text_to_anonymize = text_container.get_text()

            # Analyze the text using the analyzer engine
            analyzer_results = analyzer.analyze(text=text_to_anonymize, language='en')
 
            if text_to_anonymize.isspace() == False:
                print(text_to_anonymize)
                print(analyzer_results)

            characters = list([])

            # Grab the characters from the PDF
            for text_line in filter(lambda t: isinstance(t, LTTextLine), text_container):
                    for character in filter(lambda t: isinstance(t, LTChar), text_line):
                            characters.append(character)


            # Slice out the characters that match the analyzer results.
            for result in analyzer_results:
                start = result.start
                end = result.end
                analyzed_character_sets.append({"characters": characters[start:end], "result": result})

My name is John Doe and my email is johndoe@example.com.

[type: EMAIL_ADDRESS, start: 36, end: 55, score: 1.0, type: PERSON, start: 11, end: 19, score: 0.85, type: URL, start: 44, end: 55, score: 0.5]
I live at 123 Main St, Anytown, USA.

[type: LOCATION, start: 23, end: 30, score: 0.85, type: LOCATION, start: 32, end: 35, score: 0.85]
My phone number is 555-123-4567.

[type: PHONE_NUMBER, start: 19, end: 31, score: 0.75]
I was born on January 1, 1980.

[type: DATE_TIME, start: 14, end: 29, score: 0.85]
My driver's license number is D1234567.

[type: US_DRIVER_LICENSE, start: 30, end: 38, score: 0.6499999999999999]
My passport number is 123456789.

[type: DATE_TIME, start: 22, end: 31, score: 0.85, type: US_PASSPORT, start: 22, end: 31, score: 0.4, type: US_BANK_NUMBER, start: 22, end: 31, score: 0.05, type: US_DRIVER_LICENSE, start: 22, end: 31, score: 0.01]
My social security number is 123-45-6789.

[]
My credit card number is 1234 5678 9012 3456.

[]
My medical record number is MR1

## Create phrase bounding boxes

The next task is to take the character data, and inflate it into full phrase bounding boxes.

For example, for an email address, we'll turn the bounding boxes for each character in the email address into one single bounding box.

In [4]:
# Combine the bounding boxes into a single bounding box.
def combine_rect(rectA, rectB):
    a, b = rectA, rectB
    startX = min( a[0], b[0] )
    startY = min( a[1], b[1] )
    endX = max( a[2], b[2] )
    endY = max( a[3], b[3] )
    return (startX, startY, endX, endY)

analyzed_bounding_boxes = []

# For each character set, combine the bounding boxes into a single bounding box.
for analyzed_character_set in analyzed_character_sets:
    completeBoundingBox = analyzed_character_set["characters"][0].bbox
    
    for character in analyzed_character_set["characters"]:
        completeBoundingBox = combine_rect(completeBoundingBox, character.bbox)
    
    analyzed_bounding_boxes.append({"boundingBox": completeBoundingBox, "result": analyzed_character_set["result"]})

## Add highlight annotations

We finally iterate through all the analyzed bounding boxes and create highlight annotations for all of them.

In [7]:
pdf = Pdf.open("./Demodata_PIIText.pdf")

annotations = []

# Create a highlight annotation for each bounding box.
for analyzed_bounding_box in analyzed_bounding_boxes:

    boundingBox = analyzed_bounding_box["boundingBox"]

    # Create the annotation. 
    # We could also create a redaction annotation if the ongoing workflows supports them.
    highlight = Dictionary(
        Type=Name.Annot,
        Subtype=Name.Highlight,
        QuadPoints=[boundingBox[0], boundingBox[3],
                    boundingBox[2], boundingBox[3],
                    boundingBox[0], boundingBox[1],
                    boundingBox[2], boundingBox[1]],
        Rect=[boundingBox[0], boundingBox[1], boundingBox[2], boundingBox[3]],
        C=[1, 0, 0],
        CA=0.5,
        T=analyzed_bounding_box["result"].entity_type,
    )
    
    annotations.append(highlight)

# Add the annotations to the PDF.
pdf.pages[0].Annots = pdf.make_indirect(annotations)

# And save.
pdf.save("./Annotated_Demodata_PIIText.pdf")

## Result

The output from the samples above creates a new PDF. This contains the original content, with text highlight annotations where the PII has been found.

Each text annotation contains the name of the entity found.

## Note

Before relying on this methodology to detect or markup PII from a PDF, please be aware of the following:

### Text extraction

We purposely use a different library specifically for extracting text from the PDF. This is because text extraction is hard to get right, and it's worth using a library specifically developed with the purpose in mind.

For more details, see:

[https://pdfminersix.readthedocs.io/en/latest/topic/converting_pdf_to_text.html](https://pdfminersix.readthedocs.io/en/latest/topic/converting_pdf_to_text.html)

That said, even with a purpose built library, there may be occasions where PII is present and visible in a PDF, but it is not detected by the sample code.

This includes, but is not limited to:

- Text that cannot be reliable extracted to be analyzed. (e.g. incorrect spacing, wrong reading order)
- Text present in previous iterations of the PDF which is hidden from text extraction. (See incremental editing)
- Text present in images. (requires OCRing)
- Text present in annotations.